In [ ]:
# Uncomment the below command if you don't have openpyxl
#!pip install openpyxl 

In [1]:
import pandas as pd
import math
import seaborn as sns
import matplotlib.pyplot as plt
from collections import OrderedDict

In [2]:
market_data = pd.read_excel("../../raw_data/market_data.xlsx")
market_data.drop(index=market_data.index[0], axis=0, inplace=True)
market_data = market_data.reset_index(drop = True)

In [3]:
market_data.head(5)

,Time (UTC+10),Regions NSW Trading Price ($/MWh),Regions SA Trading Price ($/MWh),Regions TAS Trading Price ($/MWh),Regions VIC Trading Price ($/MWh),Regions NSW Trading Total Intermittent Generation (MW),Regions SA Trading Total Intermittent Generation (MW),Regions TAS Trading Total Intermittent Generation (MW),Regions VIC Trading Total Intermittent Generation (MW),Regions NSW Operational Demand (MW),Regions SA Operational Demand (MW),Regions TAS Operational Demand (MW),Regions VIC Operational Demand (MW)
0,2018-01-01 00:30:00,91.86,107.17,92.28,92.46,0.15,43.07,118.73,131.68,6974,1359,1082,4398
1,2018-01-01 01:00:00,88.83,103.31,87.53,87.62,0.13,41.67,110.48,119.98,6790,1316,1071,4238
2,2018-01-01 01:30:00,73.62,88.20,76.29,73.08,0.14,42.15,120.09,123.86,6536,1240,1067,4112
3,2018-01-01 02:00:00,71.49,85.24,75.10,70.18,0.16,38.31,114.64,132.72,6339,1194,1061,3956
4,2018-01-01 02:30:00,69.27,81.75,72.92,67.43,0.16,33.39,112.90,120.73,6195,1163,1060,3833


### Add Period as a column

In [4]:
# Create a period for a whole day which are 48 as
# Spot prices are taken by the 30 minutes mark.
period = []
count = 1
for i in range(1, len(market_data) + 1):
    period.append(count)
    count += 1
    if (i % 48) == 0:
        count = 1
        
market_data['period'] = pd.Series(period)
market_data['Time (UTC+10)'] = pd.to_datetime(market_data['Time (UTC+10)'])

### Battery Class

In [46]:
###################################
# A Class to store battery functionaly such as revenue, charge and discharge
# periods, charge and discharge spot prices, charge and discharge market dispatch
#
# NOTE : IF THERE IS ANY CHANGE IN CALCULATION, CHANGE CALCULATION MARKED BY (BATTERY CALCULATION).
#
# Consists of:
#      - charge_period : period when it should charge.
#      - discharge_period : period when it should discharge.
#      - charge_price : spot price given charging period.
#      - discharge_price : spot price given discharging period.
#      - charge_market_dispatch : set amount of market dispatch given charging period.
#      - discharge_market_dispatch : set amount of market dispatch given discharging period.
# 
# Functions:
#      - ComputeRevenue: To calculate revenue given discharge and charge period pairs.
#      - Setting : Set all the battery functionalities. (BATTERY CALCULATION)
#      - FirstOptimisation : Ensure that energy are not wasted or not used. (BATTERY CALCULATION)
#      - SecondOptimisation : Ensure that to always charge the highest price less and discharge
#                             highest price more. (BATTERY CALCULATION)
#
# Created by: Gilbert
###################################
class Battery:
    # Battery Specifications
    mlf = 0.991                 # Marginal Loss Factor
    battery_capacity = 580      # Battery Capacity
    battery_power = 300         # Battery Power
    charge_efficiency = 0.9     # Charge Efficiency
    discharge_efficiency = 0.9  # Discharge Efficiency
        
    def __init__(self, charge_period, charge_spot_price,
                 discharge_period, discharge_spot_price): 
        # Charge and Discharge Period
        self.charge_period = charge_period
        self.discharge_period = discharge_period
        
        # Spot Price during Charge and Discharge
        self.charge_price = charge_spot_price
        self.discharge_price = discharge_spot_price
    
    ########################################################################################    
                
    def Revenue(self):
        if self.charge_market_dispatch == [] or self.discharge_market_dispatch == []:
            return 0
        
        # Spot Prices
        charge_sp = np.array(self.charge_price)[:, 1]
        discharge_sp = np.array(self.discharge_price)[:, 1]
        
        # Market Dispatches
        charge_md = np.array(self.charge_market_dispatch).T
        discharge_md = np.array(self.discharge_market_dispatch).T
        
        # Revenues
        charge_revenue = (charge_sp @ charge_md) * (1 / self.mlf)
        discharge_revenue = (discharge_sp @ discharge_md) * (self.mlf)
        
        return discharge_revenue + charge_revenue
    
    ########################################################################################
    
    def Setting(self):
        
        OPENING = 0
        CLOSING = 1
        
        battery_power = self.battery_power
        battery_cap = self.battery_capacity
        
        len_charge = len(self.charge_period)
        len_discharge = len(self.discharge_period)
        
        self.charge_raw_power = ['' for i in range(len_charge)]
        self.discharge_raw_power = ['' for i in range(len_discharge)]
        
        self.charge_market_dispatch = ['' for i in range(len_charge)]
        self.discharge_market_dispatch = ['' for i in range(len_discharge)]
        
        self.charge_capacity = [[0, 0] for i in range(len_charge)]
        self.discharge_capacity = [[0, 0] for i in range(len_discharge)]
        
        # CHARGE PERIOD --------------------------------------------------------------------
        for t in range(len_charge):
            # RAW_POWER[t] = -MIN(BATTERY_POWER, (BATTERY_CAPACITY - OPENING_CAPACITY[t]) / CHARGE_EFFICIENCY * 2)
            self.charge_raw_power[t] = -min(battery_power, 
                                           (battery_cap - self.charge_capacity[t][OPENING]) / 
                                            self.charge_efficiency * 2)

            # MARKET_DISPATCH[t] = RAW_POWER / 2
            self.charge_market_dispatch[t] = self.charge_raw_power[t] / 2

            # CLOSING_CAPACITY[t] = MAX(0, MIN(OPENING_CAPACITY[t] - 
            #                        MARKET_DISPATCH[t] * CHARGE_EFFICIENCY, BATTERY_CAPACITY))
            self.charge_capacity[t][CLOSING] = max(0, min(self.charge_capacity[t][OPENING] - 
                                                        self.charge_market_dispatch[t] * self.charge_efficiency, 
                                                        battery_cap))

            # Ensuring that it doesn't exceeds array len limit
            if t + 1 < len_charge:
                self.charge_capacity[t + 1][OPENING] = self.charge_capacity[t][CLOSING]

        # DISCHARGE PERIOD -----------------------------------------------------------------

        # Set DISCHARGE CAPACITY AT t = 0 to be the the last t of CHARGING CAPACITY
        self.discharge_capacity[0][OPENING] = self.charge_capacity[-1][CLOSING]

        for t in range(len_discharge):
            # RAW_POWER[t] = MIN(BATTERY_POWER, OPENING_CAPACITY[t] * 2)
            self.discharge_raw_power[t] = min(battery_power, self.discharge_capacity[t][OPENING] * 2)

            # MARKET_DISPATCH[t] = RAW_POWER[t] / 2 * DISCHARGE EFFICIENCY
            self.discharge_market_dispatch[t] = self.discharge_raw_power[t] / 2 * self.discharge_efficiency

            # CLOSING CAPACITY[t] = MAX(0, MIN(OPENING_CAPACITY[t] -
            #                        MARKET_DISPATCH[t] * (1/DISCHARGE_EFFICIENCY), BATTERY_CAPACITY))
            self.discharge_capacity[t][CLOSING] = max(0, min(self.discharge_capacity[t][OPENING] -
                                                            self.discharge_market_dispatch[t] * (1 / self.discharge_efficiency),
                                                            battery_cap))
            # Ensuring that it doesn't exceeds array len limit
            if t + 1 < len_discharge:
                self.discharge_capacity[t + 1][OPENING] = self.discharge_capacity[t][CLOSING]
    
    ########################################################################################
        
    def FirstOptimisation(self):
        OPENING = 0
        CLOSING = 1
        
        battery_power = self.battery_power
        battery_cap = self.battery_capacity
        
        len_charge = len(self.charge_period)
        len_discharge = len(self.discharge_period)
        
        MAX_CHARGE_PERIOD = 5
        MAX_DISCHARGE_PERIOD = 4
        
        if (len_charge >= MAX_CHARGE_PERIOD and len_discharge >= MAX_DISCHARGE_PERIOD) or (len_charge == len_discharge):
            #md = 117
            #self.discharge_raw_power[-1] = min(md * 2 / self.discharge_efficiency, self.discharge_raw_power[-1])
            #self.discharge_market_dispatch[-1] = self.discharge_raw_power[-1] / 2 * self.discharge_efficiency
            return
        elif (len_charge - len_discharge == 1):
            self.discharge_capacity[-1][CLOSING] = 0
            # OPENING_CAPACITY[-1] = MARKET_DISPATCH / DISCHARGE_EFFICIENCY
            self.discharge_capacity[-1][OPENING] = self.discharge_market_dispatch[-1] / self.discharge_efficiency
            for t in range(1, len_discharge):
                # CLOSING_CAPACITY[-t - 1] = OPENING_CAPACITY[-t]
                self.discharge_capacity[-t - 1][CLOSING] = self.discharge_capacity[-t][OPENING]
                # OPENING_CAPACITY[-t - 1] = CLOSING_CAPACITY[-t - 1] + MARKET_DISPATCH[-t - 1] / DISCHARGE_EFFICIENCY
                self.discharge_capacity[-t - 1][OPENING] = self.discharge_capacity[-t - 1][CLOSING] + self.discharge_market_dispatch[-t - 1] / self.discharge_efficiency
            
            # CLOSING_CAPACITY[-1] = OPENING_CAPACITY[0]
            self.charge_capacity[-1][CLOSING] = self.discharge_capacity[0][OPENING]
            # MARKET_DISPATCH[-1] = -(CLOSING_CAPACITY[-1] - OPENING_CAPACITY[-1]) / CHARGE_EFFICIENCY
            self.charge_market_dispatch[-1] = -(self.charge_capacity[-1][CLOSING] - self.charge_capacity[-1][OPENING]) / self.charge_efficiency
            # RAW_POWER[-1] = MARKET_DISPATCH[-1] * 2
            self.charge_raw_power[-1] = self.charge_market_dispatch[-1] * 2
            
    ########################################################################################

    def SecondOptimisation(self):
        OPENING = 0
        CLOSING = 1
        
        battery_power = self.battery_power
        battery_cap = self.battery_capacity
        
        len_charge = len(self.charge_period)
        len_discharge = len(self.discharge_period)
        
        MAX_CHARGE_PERIOD = 5
        MAX_DISCHARGE_PERIOD = 4
        
        # CHARGE PERIOD --------------------------------------------------------------------
        highest_price_index = np.array(self.charge_period).argmax(axis=0)[0]
        lowest_charge_index = self.charge_market_dispatch.index(max(self.charge_market_dispatch))    
        
        # IF THE HIGHEST CHARGING PRICE DOESN'T HAVE THE LOWEST CHARGING RATE, SWAP!
        if (highest_price_index != lowest_charge_index):
            tmp = self.charge_market_dispatch[highest_price_index]
            self.charge_market_dispatch[highest_price_index] = self.charge_market_dispatch[lowest_charge_index]
            self.charge_market_dispatch[lowest_charge_index] = tmp
            # SET THE UPDATED BATTERY SETTINGS.
            for t in range(len_charge):
                # RAW_POWER[t] = MARKET_DISPATCH[t] * 2
                self.charge_raw_power[t] = self.charge_market_dispatch[t] * 2

                # CLOSING_CAPACITY[t] = MAX(0, MIN(OPENING_CAPACITY[t] - 
                #                        MARKET_DISPATCH[t] * CHARGE_EFFICIENCY, BATTERY_CAPACITY))
                self.charge_capacity[t][CLOSING] = max(0, min(self.charge_capacity[t][OPENING] - 
                                                            self.charge_market_dispatch[t] * self.charge_efficiency, 
                                                            battery_cap))
                # Ensuring that it doesn't exceeds array len limit
                if t + 1 < len_charge:
                    self.charge_capacity[t + 1][OPENING] = self.charge_capacity[t][CLOSING]

        # DISCHARGE PERIOD -----------------------------------------------------------------
        lowest_price_index = np.array(self.discharge_period).argmax(axis=0)[0] 
        min_discharge_index = self.discharge_market_dispatch.index(min(self.discharge_market_dispatch))   
        
        # IF THE LOWEST DISCHARGING PRICE DOESN'T HAVE THE LOWEST DISCHARGING RATE, SWAP!
        if (lowest_price_index != min_discharge_index):
            tmp = self.discharge_market_dispatch[lowest_price_index]
            self.discharge_market_dispatch[lowest_price_index] = self.discharge_market_dispatch[min_discharge_index]
            self.discharge_market_dispatch[min_discharge_index] = tmp
            # SET THE UPDATED BATTERY SETTINGS.
            for t in range(len_discharge):
                # RAW_POWER[t] = MARKET_DISPATCH[t] * 2 / DISCHARGE_EFFICIENCY
                self.discharge_raw_power[t] = self.discharge_market_dispatch[t] * 2 / self.discharge_efficiency

                # CLOSING_CAPACITY[t] = MAX(0, MIN(OPENING_CAPACITY[t] - 
                #                        MARKET_DISPATCH[t] * CHARGE_EFFICIENCY, BATTERY_CAPACITY))
                self.discharge_capacity[t][CLOSING] = max(0, min(self.discharge_capacity[t][OPENING] - 
                                                            self.discharge_market_dispatch[t] / self.discharge_efficiency, 
                                                            battery_cap))
                # Ensuring that it doesn't exceeds array len limit
                if t + 1 < len_discharge:
                    self.discharge_capacity[t + 1][OPENING] = self.discharge_capacity[t][CLOSING]
        

### Helper function for battery optimisation

In [47]:
###################################
# A function to get spot prices based on selected regions.
#
# Parameters:
#      - data : the targeted dataset, minimum dataset length of 48.
#      - selected_periods : selected period for charging or discharging.
#      - region : the targeted region, default has been set to 'VIC' for mandatory task.
#
# Return:
#      - List of spot prices given period
#
# Created by: Gilbert
###################################
def GetSpotPrice(data, selected_periods, region = 'VIC'):
    if region == 'VIC':
        spot_price = data['Regions VIC Trading Price ($/MWh)']
    elif region == 'NSW':
        spot_price = data['Regions NSW Trading Price ($/MWh)']
    elif region == 'SA':
        spot_price = data['Regions SA Trading Price ($/MWh)']
    elif region == 'TAS':
        spot_price = data['Regions TAS Trading Price ($/MWh)']
    
    spot_price = np.array(spot_price)
    
    # Find the spot prices from selected region. Periods are
    # index + 1, therefore to use the index we need to subtract
    # it by 1.
    retrieved_prices = []
    for period in selected_periods:
        # append(minimum or maximum ranking, spot_price[index])
        retrieved_prices.append((period[1], spot_price[period[1] - 1]))
    return retrieved_prices

########################################################################################

###################################
# A function to find minimum and maximum point rank given threshold.
#
# Parameters:
#      - data : the targeted dataset, minimum dataset length of 48.
#      - region : the targeted region, default has been set to 'VIC' for mandatory task.
#      - buy_threshold : maximum number of buying point, default has been set to optimise Checkpoint 3.
#      - sell_threshold : maximum number of selling point, default has been set to optimise Checkpoint 3.
#
# Return:
#      - List of selected minimum point, list of selected maximum point
#
# Efficiency: O(3N + NLogN) = O(NLogN)
#
# Created by: Gilbert
###################################
def GetMinMax(data, region = 'VIC', buy_threshold = 5, sell_threshold = 4):
    EMPTY = ''
    if region == 'VIC':
        spot_price = data['Regions VIC Trading Price ($/MWh)']
    elif region == 'NSW':
        spot_price = data['Regions NSW Trading Price ($/MWh)']
    elif region == 'SA':
        spot_price = data['Regions SA Trading Price ($/MWh)']
    elif region == 'TAS':
        spot_price = data['Regions TAS Trading Price ($/MWh)']
    
    price = np.array(spot_price)
    minimum_price = np.argsort(price, kind = 'merge*sort') # (O(NlogN)), mergesort the minimum prices.
    maximum_price = minimum_price[::-1][:48] # (O(N)), maximum is the reverse order of minimum.
    
    selected_min_price = [EMPTY for i in minimum_price] # (O(N)), set an empty array for the whole period.
    selected_max_price = [EMPTY for i in minimum_price] # (O(N)), set an empty array for the whole period.
    
    # Select the lowest price spot over the given
    # buy_threshold as the minimum buying point.
    i = 0
    for b_t in range(buy_threshold):
        selected_min_price[minimum_price[i]] = b_t + 1
        i += 1
        
    # Select the highest price spot over the given
    # sell_threshold as the maximum selling point.
    i = 0
    for s_t in range(sell_threshold):
        selected_max_price[maximum_price[i]] = s_t + 1
        i += 1
        
    return selected_min_price, selected_max_price

########################################################################################

###################################
# A function to Find Battery Charge and Discharge pairs in backward order.
# Backward order from 48th period to the 1st.
#
# Parameters:
#      - buy_period : Selected minimum price point as it will be where we buy energy for charging.
#      - sell_period : Selected maximum price point as it will be where we sell energy for discharging.
#
# Return:
#      - List of battery class pairs
#
# Efficiency: O(N)
#
# Created by: Gilbert
###################################
def FindBatteryPairs(buy_period, sell_period):
    MAX_SELL_PERIOD = 4 # MAXIMUM SELLING PERIOD PER PAIR
    MAX_BUY_PERIOD = 5 # MAXIMUM BUYING PERIOD PER PAIR
    EMPTY = ''
    period = 48
    battery = []
    sell = OrderedDict() # Initialise battery selling point. (Ordered Dictionary)
    buy = OrderedDict() # Initialise battery buying point. (Orderered Dictionary)
    
    # Iterate over the whole period backwards
    for p in range(period - 1, -1, -1):
        # If maximum selling point is not empty, add (order, period)
        # as key-value pair into the OrderedDict.
        if sell_period[p] != EMPTY:
            # If battery buying point period is less MAXIMUM SELLING 
            # PERIOD PER PAIR, add new period.
            if len(sell) < MAX_SELL_PERIOD:
                sell[sell_period[p]] = sell_period.index(sell_period[p]) + 1
            # else, if battery selling point is full and there is 
            # higher maximum selling point then remove the lowest
            # selling point and add the new one into Dictionary.    
            else:
                max_key = max(sell, key=int)
                if sell_period[p] < max_key:
                    sell.pop(max_key)
                    sell[sell_period[p]] = sell_period.index(sell_period[p]) + 1

        # If battery selling point is not empty and minimum buying 
        # point is not empty.
        if len(sell) != 0 and buy_period[p] != EMPTY:
            # If battery buying point period is less MAXIMUM BUYING 
            # PERIOD PER PAIR, add new period.
            if len(buy) < MAX_BUY_PERIOD and len(buy) < math.ceil(len(sell) * 1.25):
                buy[buy_period[p]] = buy_period.index(buy_period[p]) + 1
            # else, if battery buying point is full and there is 
            # lower minimum buying point then remove the highest
            # buying point and add the new one into Dictionary.
            else:
                max_key = max(buy, key=int)
                if buy_period[p] < max_key:
                    buy.pop(max_key)
                    buy[buy_period[p]] = buy_period.index(buy_period[p]) + 1
        # If the next period is not empty and battery buying point
        # is not empty then battery charge-discharge pair has been
        # created.
        # Reinitialise a new battery setup.
        if sell_period[p - 1] != EMPTY and len(buy) != 0:
            battery.append([list(sell.items()), list(buy.items())])
            sell = OrderedDict()
            buy = OrderedDict()
    # Add the last battery charge-discharge pair occuring 
    # before 1st period.
    battery.append([list(sell.items()), list(buy.items())]) 

    # Check whether there is too many selling points, then
    # remove selling point until the number of selling points
    # is equal to the number of buying points while removing
    # the lowest selling point.
    for b in battery:
        sell_tmp = np.array(b[0])
        buy_tmp = b[1]
        while len(sell_tmp) > len(buy_tmp): 
            row = 0
            index = np.where(sell_tmp[:,0] == sell_tmp[:,0].max())[0][0]
            sell_tmp = np.delete(sell_tmp, index, axis = row)
        b[0] = sell_tmp.tolist() # Change numpy array to list
        
    return battery

########################################################################################

###################################
# A function to set optimal charge and discharge amount of battery pairs.
#
# Parameters:
#      - data : the targeted dataset, minimum dataset length of 48.
#      - battery_pairs : list of all battery class pairs.
#
# Return:
#      - List of all battery class pairs
#
# Efficiency: O(N)
#
# Created by: Gilbert
###################################
def SetChargeDischarge(data, battery_pairs, region = "VIC"):    
    all_batteries = []
    
    battery_pairs = battery_pairs[::-1]
    for b in battery_pairs:
        sell_period = b[0][::-1] # Reverse the order
        buy_period = b[1][::-1] # Reverse the order
        if len(sell_period)!= 0 or len(buy_period) != 0:
            sell_price = GetSpotPrice(data, sell_period, region = region)
            buy_price = GetSpotPrice(data, buy_period, region = region)

            battery = Battery(buy_period, buy_price, sell_period, sell_price)
            battery.Setting()
            battery.FirstOptimisation()
            battery.SecondOptimisation()

            all_batteries.append(battery)

    return all_batteries

########################################################################################

###################################
# A function to calculate daily revenue.
#
# Parameters:
#      - all_batteries : List of battery class pairs.
#
# Return:
#      - Daily revenues
#
# Created by: Gilbert
###################################
def ComputeDailyRevenue(all_batteries):
    revenues = 0
    for battery in all_batteries:
        revenues += battery.Revenue()
    return revenues

########################################################################################

###################################
# A function to optimise battery charging and discharging period. This is where 
# mainly the optimisations are performed with the helper functions.
#
# Parameters:
#      - daily_data : the targeted daily dataset, minimum dataset length of 48.
#
# Return:
#      - raw_power : List of Daily Raw Power for charging and discharging
#      - market_dispatch : List of Daily Market Dispatch for charging and discharging
#      - opening_capacity : List of Daily Opening Capacity for charging and discharging
#      - closing_capacity : List of Daily Closing Capacity for charging and discharging
#
# Efficiency: O(N^2 * NLogN) = O(N^3LogN)
# Created by: Gilbert
###################################
def DailyOptimisation(daily_data, region = "VIC"):
    best_batteries = OrderedDict() # Initialise an Ordered Dictionary
    
    # TODO: OPTIMISE EFFICIENCY HERE! Make this at least < N^2
    # Iterate over all possible combinations of battery pairs. (O(N^2))
    for s in range(1, 49):
        for b in range(1, 49):
            # Get the minimum and maximum price based on the given threshold
            min_price, max_price = GetMinMax(daily_data, buy_threshold = b, sell_threshold = s, region = region)
            # Get the battery pairs based on minimum and maximum price
            battery_pairs = FindBatteryPairs(min_price, max_price)
            # Get battery optimisation for the selected threshold
            all_batteries = SetChargeDischarge(daily_data, battery_pairs, region = region)
            # Compute daily revenues of selected battery combinations
            dailyrev = ComputeDailyRevenue(all_batteries)

            if dailyrev < 0:
                break

            # Insert revenue as key, batteries combination and threshold as value
            if dailyrev not in best_batteries:
                best_batteries[dailyrev] = (all_batteries, (b, s))
                
    # Find the highest revenue amongst possible combinations in that day
    best_revenue = max(best_batteries)
    best_threshold = best_batteries[best_revenue][1]
    #print(best_threshold)
    battery = best_batteries[best_revenue][0] # The Best battery combinations
    
    # Initialise raw_power, market_dispatch, opening_capacity and closing capacity
    raw_power = [0 for i in range(48)]
    market_dispatch = [0 for i in range(48)]
    opening_capacity = [0 for i in range(48)]
    closing_capacity = [0 for i in range(48)]
    
    # Iterate over battery combinations to set raw_power, market_dispatch,
    # opening_capacity, closing_capacity into an array to be prepared for 
    # merging with the dataset.
    for b in battery:   
        # Charging Period
        for cp in range(len(b.charge_period)):
            raw_power[b.charge_period[cp][1] - 1] = b.charge_raw_power[cp]
            market_dispatch[b.charge_period[cp][1] - 1] = b.charge_market_dispatch[cp]
            opening_capacity[b.charge_period[cp][1] - 1] = b.charge_capacity[cp][0]
            closing_capacity[b.charge_period[cp][1] - 1] = b.charge_capacity[cp][1]
        # Discharge Period
        for dp in range(len(b.discharge_period)):
            raw_power[b.discharge_period[dp][1] - 1] = b.discharge_raw_power[dp]
            market_dispatch[b.discharge_period[dp][1] - 1] = b.discharge_market_dispatch[dp]
            opening_capacity[b.discharge_period[dp][1] - 1] = b.discharge_capacity[dp][0]
            closing_capacity[b.discharge_period[dp][1] - 1] = b.discharge_capacity[dp][1]
    
    # Formatting the opening and closing capacity.
    for i in range(1, len(opening_capacity)):
        if closing_capacity[i - 1] != 0.0 and opening_capacity[i] == 0.0:
            opening_capacity[i] = closing_capacity[i - 1]
            closing_capacity[i] = opening_capacity[i]
    
    return raw_power, market_dispatch, opening_capacity, closing_capacity

In [48]:
import time

# Test Period checkpoint 3
cp3_start_period = '2020-07-17 00:30:00'
cp3_end_period   = '2020-07-18 00:00:00'

vic_spot_price = market_data[['Time (UTC+10)', 'period', 'Regions VIC Trading Price ($/MWh)']]

cp_3 = vic_spot_price.loc[(vic_spot_price['Time (UTC+10)'] >= cp3_start_period) & \
        (vic_spot_price['Time (UTC+10)'] <= cp3_end_period)]

cp_3.to_csv('../../preprocessed_data/checkpoint3.csv', index = False)

check3 = pd.read_csv('../../preprocessed_data/checkpoint3.csv')

tic = time.perf_counter()
raw_power, market_dispatch, opening_capacity, closing_capacity = DailyOptimisation(check3)
toc = time.perf_counter()
print(f"Daily CV Duration is: {toc - tic:0.4f}")
check3['Raw Power'] = pd.Series(raw_power)
check3['Market Dispatch'] = pd.Series(market_dispatch)
check3['Opening Capacity'] = pd.Series(opening_capacity)
check3['Closing Capacity'] = pd.Series(closing_capacity)
#check3['minima'] = pd.Series(minima)
#check3['maxima'] = pd.Series(maxima)
#validation = check3[['Time (UTC+10)', 'Raw Power', 'Opening Capacity']]
#validation.columns = ['datetime', 'power', 'capacity']
check3

Daily CV Duration is: 0.7844


,Time (UTC+10),period,Regions VIC Trading Price ($/MWh),Raw Power,Market Dispatch,Opening Capacity,Closing Capacity
0,2020-07-17 00:30:00,1,75.51,0.000000,0.000000,0.0,0.0
1,2020-07-17 01:00:00,2,73.98,0.000000,0.000000,0.0,0.0
2,2020-07-17 01:30:00,3,75.57,0.000000,0.000000,0.0,0.0
3,2020-07-17 02:00:00,4,71.94,0.000000,0.000000,0.0,0.0
4,2020-07-17 02:30:00,5,74.10,0.000000,0.000000,0.0,0.0
5,2020-07-17 03:00:00,6,67.36,0.000000,0.000000,0.0,0.0
6,2020-07-17 03:30:00,7,58.04,-300.000000,-150.000000,0.0,135.0
7,2020-07-17 04:00:00,8,51.85,-300.000000,-150.000000,135.0,270.0
8,2020-07-17 04:30:00,9,74.53,0.000000,0.000000,270.0,270.0
9,2020-07-17 05:00:00,10,67.32,0.000000,0.000000,270.0,270.0


In [ ]:
check3.to_csv('../../preprocessed_data/checkpoint3_optimised.csv', index = False)
#validation.to_csv('../../preprocessed_data/checkpoint3_validation.csv', index = False)

In [ ]:
raw_power = []
market_dispatch = []
opening_capacity = []
closing_capacity = []

threshold = []

period = 48

start = 0
end = period 

tic = time.perf_counter()
while end <= len(market_data):
    daily_data = market_data.iloc[start:end, :]
    daily_rp, daily_md, daily_oc, daily_cc = DailyOptimisation(daily_data)
    raw_power.extend(daily_rp)
    market_dispatch.extend(daily_md)
    opening_capacity.extend(daily_oc)
    closing_capacity.extend(daily_cc)

    start += period
    end += period
    
toc = time.perf_counter()
print(f"Total Training CV Duration is: {toc - tic:0.4f}")

market_data['Raw Power (MW)'] = pd.Series(raw_power)
market_data['Market Dispatch (MWh)'] = pd.Series(market_dispatch)
market_data['Opening Capacity (MWh)'] = pd.Series(opening_capacity)
market_data['Closing Capacity (MWh)'] = pd.Series(closing_capacity)

In [ ]:
vic_data = market_data[['Time (UTC+10)', 'period', 'Regions VIC Trading Price ($/MWh)', 'Raw Power (MW)', 
                        'Market Dispatch (MWh)', 'Opening Capacity (MWh)', 'Closing Capacity (MWh)']]
vic_data.to_excel('../../preprocessed_data/Victoria_data_v1.xlsx', index = False)